# Lecture 25 Deploying Data Science Projects as Web Applications <a id="section0"/>

- [25.1 Introduction to Flask](#section1)
- [25.2 Deploying a Model for Iris Flowers Classification](#section2)
    - [25.2.1 Model Training and Prediction](#section3)
    - [25.2.2 Creating an HTML Webpage](#section4)
    - [25.2.3 Creating a Flask App](#section5)
    - [25.2.4 Output Webpage](#section6)
    - [25.2.5 Run the Web Application](#section7)
- [25.3 Deploying a Model for MNIST Digits Classification](#section8)
    - [25.3.1 Model Training and Saving](#section9)
    - [25.3.2 Creating a Flask App](#section10)
    - [25.3.3 Run the Web Application](#section11)
- [References](#section12)


# 25.1 Introduction to Flask <a id="section1"/>

So far in this course we learned about Python programming, data engineering pipelines, and model engineering pipelines. The next theme focuses on model deployment pipelines. Once we know how to use Machine Learning models for solving data science tasks, we will often need to deploy the models in a production environment, so that they can be used by others. In this lecture, we will learn how to build a simple web API service for deploying models to websites. 

Several libraries have been developed for integrating Machine Learning models into web pages. In this lecture we will learn how to use Flask for this purpose. ***Flask*** is a Python-based micro framework for developing web APIs and it enables deploying Python applications as web services. 

Flask allows a Python program to send and receive messages to/from a web page. Flask uses GET and POST methods to access the web page, where GET accesses information from the web page (e.g., based on users' inputs), and POST sends information to the web page (e.g., post the model predictions).

Other similar frameworks for deploying ML models as web applications include Django, Cortex, MLFlow, BentoML, as well as there are many frameworks developed by the cloud providers, such as Amazon SageMaker, Azure ML, Google Cloud ML, IBM Watson Cloud. Flask is easier to learn and use in comparison to other frameworks, but it also has limitations in terms of functionality (e.g., Flask is designed for low-volume API, and does not scale well to large applications).  

Flask is not typically run from Jupyter Notebooks, and therefore we will use python scripts from the command line in a terminal.

# 25.2 Deploying a Model for Iris Flowers Classification <a id="section2"/>

This section was inspired by the article [Deploying Machine Learning Models using Flask](https://www.section.io/engineering-education/deploying-machine-learning-models-using-flask/).

The goal is to deploy a simple Machine Learning model for classification of the Iris dataset, which we used in Lecture 12. Recall that the dataset consists of measurements of three different species of irises:
  1. Iris Setosa
  2. Iris Versicolour
  3. Iris Virginica
  
Each data point has 4 features, which include:
  1. Sepal length in cm
  2. Sepal width in cm
  3. Petal length in cm
  4. Petal width in cm
  
Let's load the dataset using Pandas, and display the dataframe.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

# Load the dataset
data = pd.read_csv('Iris-API/iris.csv')

In [2]:
data

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In the dataframe, the `variety` column contains the target labels. First, we will convert the labels to numerical values, so that we can fit them to a classification model.

In [3]:
# Encoding the target variables to integers
data = data.replace(['Setosa', 'Versicolor' , 'Virginica'],[0, 1, 2])

data

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


## 25.2.1 Model Training and Prediction <a id="section3"/>

Let's split the data into training data variable `X` and training labels variable `y`. We will use a Logistic Regression model from scikit-learn to train a classification model. 

In [4]:
# Create training data and labels
X = data.iloc[:, 0:-1] 
y = data.iloc[:, -1] 

logreg = LogisticRegression() 
# Fitting the model
logreg.fit(X, y) 

LogisticRegression()

Finally, let's define a function `classify` that takes 4 input arguments, and predicts the class of the iris. The four inputs `a`, `b`, `c`, and `d` correspond to the sepal length and width, and petal length and width. The function first converts the intputs to a numpy array, and reshapes it to represent a batch with 1 data instance. Afterward, the trained classifier is used to predict the class of the iris flower.

In [5]:
# Function for classification based on inputs
def classify(a, b, c, d):
    arr = np.array([a, b, c, d]) # Convert to numpy array
    arr = arr.astype(np.float64) # Change the data type to float
    query = arr.reshape(1, -1) # Reshape the array
    pred = logreg.predict(query) # Make a prediction
    prediction = variety_mappings[pred[0]] # Retrieve the name of the iris flower
    return prediction # Return the prediction

# Dictionary containing the mapping
variety_mappings = {0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'}

To make sure that that function works, in the following cell an input set is passed to `classify`.

In [6]:
classify(1.0, 1.0, 2.0, 2.0)

'Setosa'

The above code for classifying iris flowers is saved as `model.py` under the directory `IRIS-API/model.py`.

## 25.2.2 Creating an HTML Webpage<a id="section4"/>

Next, we will build a simple form using HTML to accept inputs from the user and output the iris class, shown below. 

<img style="float: center; height:300px;" src="images/home_page.png">

The HTML code for the input form is shown below. The `head` defines styles and metadata, and the `body` contains div tags with `class='field'` for each of the 4 inputs: sepal/petal lengths/widths. Each input tag has `class="input"`, and `type="number"` to accept numerical values from the users. 

All the div tags are enclosed within a form with `action="predict"` and `method="GET"`. Here, GET request is used to transport the data from the HTML form to the Flask app.

The HTML code is saved as `index.html` under the directory `IRIS-API/templates/index.html`.

<img style="float: center; height:500px;" src="images/home_head.png">
<img style="float: center; height:700px;" src="images/home_body.png">

## 25.2.3 Creating a Flask App <a id="section5"/>

Next, we will use the Flask framework to deploy the Machine Learning model locally. 

To do that, let's first import the `model` from the file that we created, and import the `flask` library and other related libraries. 

Building the Flask server begins with initializing the Flask class, which has been assigned to the name `app`. Here, we specified the current module `__name__` as an argument to the constructor `Flask()`, and we also input the `template_folder` containing the webpage-related files. 

In [ ]:
import model # Import the python file containing the ML model
from flask import Flask, request, render_template # Import flask libraries

# Initialize the flask class and specify the templates directory
app = Flask(__name__, template_folder="templates")

In the next cell, we set the default route of the server to the index page by specifying the path in `@app.route()` function. The `render_template('index.html')` method will display the script `index.html` in the browser.

In [ ]:
# Default route set as 'index'
@app.route('/')
def index_view():
    return render_template('index.html') # Render index.html

Similarly, we create a separate route for the Machine Learning model. Here, we use the `/predict` route with the methods `'POST'` and `'GET'`. This will retrieve the data from the `index.html` form through a GET request. To do that, we use `request.args.get()` to retrieve the data from each of the input fields in the form, using their name attributes.

The retrieved inputs `sepal_len`, `sepal_wid`, `petal_len`, `petal_wid` are passed to the `classify()` method of the Machine Learning model.

To render the predicted value for the variety of flowers, we specify the output HTML file along with the predicted class, using the `render_template(filename, arguments)`.

In [ ]:
# Route 'predict' accepts GET request
@app.route('/predict', methods=['POST','GET'])
def predict_class():
    try:
        sepal_len = request.args.get('slen') # Get parameters for sepal length
        sepal_wid = request.args.get('swid') # Get parameters for sepal width
        petal_len = request.args.get('plen') # Get parameters for petal length
        petal_wid = request.args.get('pwid') # Get parameters for petal width

        # Get the output from the classification model
        variety = model.classify(sepal_len, sepal_wid, petal_len, petal_wid)

        # Render the output in new HTML page
        return render_template('output.html', variety=variety)
    except:
        return 'Error'

To run the Flask webserver, we use the `app.run()` method as shown below.

In [ ]:
# Run the Flask server
if(__name__=='__main__'):
    app.run(debug=True)  

The code for Flask is saved as `app.py` under the directory `IRIS-API/app.py`. We will use it later to run the web application. 

## 25.2.4 Output Webpage <a id="section6"/>

We need to also create another even simpler HTML webpage to display the output of the classification. 

<img style="float: center; height:150px;" src="images/output_page.png">

The body of the code is shown below, and it is saved as `output.html` in the directory `./templates/output.html`. In the code `{{ variety }}` specifies the argument that was passed to get rendered in the new HTML file.

<img style="float: center; height:300px;" src="images/output_body.png">

## 25.2.5 Run the Web Application <a id="section7"/>

To deploy the application, we will execute the command `python app.py` from the terminal. Next, we copy and paste the URL `http://127.0.01:5000/` (IP Address:Port) to start the application on the local machine.

<img style="float: center; height:120px;" src="images/run_server.png">

The following are screenshots of the application for one example with a predicted iris class Setosa.

<img style="float: center; height:300px;" src="images/sample_1.png">

<img style="float: center; height:100px;" src="images/sample_2.png">

The folder structure is as follows:
```
Iris-API
    ├── iris.csv
    ├── model.py
    ├── app.py
    ├── templates
    │   ├── index.html
    │   ├── output.html
```


# 25.3 Deploying a Model for MNIST Digits Classification <a id="section8"/>

In this section we will create another web API for classification of MNIST digits. Differently from the previous section, we will use a Convolutional Neural Network model, as well as we will save the trained model and use it for prediction. The code in this section is from the following blog: [Deploying Deep Learning Models Part 1: Preparing the Model](https://blog.paperspace.com/deploying-deep-learning-models-flask-web-python/).

## 25.3.1 Model Training and Saving <a id="section9"/>

We will first load the MNIST dataset, and we will create a simple Convolutional Neural Network for classification of the digit images in the dataset.

In [7]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,  Conv2D, MaxPooling2D

In [8]:
# Loading the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [9]:
# Scale the images to the range [0,1]
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Print the shape and number of samples
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


In [10]:
# Convert the labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [13]:
# Define the model 
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [14]:
# Compile and train the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=128,
          epochs=12, verbose=0, validation_split=0.2)

In [15]:
# Evaluate on test dataset
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.8191999793052673


Next, we will save the model architecture in a JSON file, and we will save the weights of the model in an HDF5 file. We will use these files to make predictions in the web application, without retraining the model. 

In [16]:
# Save the model architecture to JSON file
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# save the model weights to HDF5 file
model.save_weights("model.h5")

The JSON file saves the information for all layers in the model. Part of it is depicted in the next figure, where the information for the second layer of the model is expanded. Similar information is saved for all layers in the model. 

<img style="float: center; height:400px;" src="images/json.png">

To make predictions, we will need to load the architecture of the model and the trained weights of the model, as in the following cell. 

In [17]:
from tensorflow.keras.models import model_from_json

# Open and store the JSON file in a variable
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()

# Use model_from_json to create a model
loaded_model = model_from_json(loaded_model_json)

# load the weights into new model
loaded_model.load_weights("model.h5")

# Compile and evaluate loaded model
loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Evaluate on test dataset
score = loaded_model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.8191999793052673


## 25.3.2 Creating a Flask App <a id="section10"/>

To deploy the model we will create the `app.py` file, similarly to the Flask file in the previous section that was named `server.py`.

We will define again two routes in the flask app:
- An index page route, for the users to draw a digit image.
- A predict route, to predict the class of the digit using the saved model.

In the next cell, we will import the required libraries, as well the function `init()` will load the saved model. 

In [ ]:
from flask import Flask, render_template,request
from PIL import Image
import numpy as np
import tensorflow.keras.models
import re
import sys 
import os
import base64
sys.path.append(os.path.abspath("./model"))
from load import * 

global model

model = init()

In [ ]:
def init(): 
    json_file = open('model/model.json','r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    #load weights into new model
    loaded_model.load_weights('model/model.h5')
    print("Loaded Model from disk")
    #compile loaded model
    loaded_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return loaded_model

In the next cell, an `app` object is created that is an instance of the Flask class. And, we define the route using the `@app.route` decorator, and pass the URL of the main page of the web application `index.html`, as in Section 25.2.3. 

In [ ]:
app = Flask(__name__)

@app.route('/')
def index_view():
    return render_template('index.html')

The predict route is shown next, which gets the image of a digit drawn by the user, and it first applies image resizing and reshaping, and afterward the model is used to predict the digit class. Output is the `response` variable that returns a string of the predicted class. 

In [ ]:
@app.route('/predict/', methods=['GET','POST'])

def predict():
    imgData = request.get_data()
    
    convertImage(imgData)
    x = Image.open('output.png')
    x = x.resize((28,28))
    x = np.invert(x)
    x = x[:,:,0].reshape(1,28,28,1)
    x = x/255

    out = model.predict(x)
    print(out)
    print(np.argmax(out,axis=1))

    response = np.array_str(np.argmax(out,axis=1))
    return response	

And finally, run the app using the local port 8000. 

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, port=8000)

The organization of the directory of the web application is shown below. Beside the `app.py`, in the main directory the image drawn by the users is saved as `output.png`. The `templates` sub-directory contains the `index.html` and `draw.html` pages. In the `static` folder, a JavaScript file `index.js` is used to render a canvas for drawing digits by the users, and `style.css` is a stylesheet for the web application. And, the `model` sub-directory contains the `load.py` file for loading the model, and `model.h5` and `model.json` files of the saved model. 

```
MNIST-API
    ├── app.py
    ├── output.png
    ├── templates
    │   ├── index.html
    │   ├── draw.html
    ├── static
    │   ├── index.js
    │   ├── style.css
    ├── model
    │   ├── load.py
    │   ├── model.h5
    │   ├── model.json
```

## 25.3.3 Run the Web Application <a id="section11"/>

We can run the application using `python app.py`, and we will copy and paste the shown URL `http://127.0.0.1:8000/` in a browser. The web API is shown below, where we can use the mouse pointer to draw digits, and the Predict button will result in a Predicted Output by the model. The Clear button will allow to draw again and make a new prediction.

<img style="float: center; height:200px;" src="images/mnist_run.png">

<img style="float: center; height:400px;" src="images/mnist_prediction.png">

# References <a id="section12"/>

1. Deploying Machine Learning Models using Flask, by Srishilesh P S, available at [https://www.section.io/engineering-education/deploying-machine-learning-models-using-flask/](https://www.section.io/engineering-education/deploying-machine-learning-models-using-flask/).
2. Deploying Deep Learning Models Part 1: Preparing the Model, by Vihar Kurama, available at: [https://blog.paperspace.com/deploying-deep-learning-models-flask-web-python/](https://blog.paperspace.com/deploying-deep-learning-models-flask-web-python/).

[BACK TO TOP](#section0)